### In this notebook I am getting data from [CPJ](https://cpj.org/data/) on the missing Journalists from 1992 to 2024.

In [9]:
#import the necessary module
import pandas as pd


In [2]:
# What did I do to get here:
# 1. I went onto the database, right-clicked and selected "inspect elements"
# 2. I went onto "Network" and I copied the undocumented API as bash
# 3. I went onto curl converter and pasted what I had copied.
# 4. The result below is what I got from Curl converter
import requests

headers = {
    'sec-ch-ua': '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Referer': 'https://cpj.org/',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
    'sec-ch-ua-platform': '"Windows"',
}

response = requests.get(
    'https://datamanager.cpj.org/api/datamanager/reports/entries?distinct(personId)&includes=organizations,fullName,location,status,typeOfDeath,charges,startDisplay,mtpage,country,type,motiveConfirmed&sort=fullName&pageNum=1&pageSize=20&in(status,%27Missing%27)&ge(year,1992)&le(year,2024)',
    headers=headers,
)

In [3]:
#if response == 200 we are good to go
response

<Response [200]>

In [4]:
# storing the response as data
data = response.json()

In [5]:
# printing the data to see if it works
data

{'rowCount': 72,
 'pageNum': 1,
 'pageSize': '20',
 'pageCount': 4,
 'data': [{'organizations': 'Agence France-Presse',
   'fullName': 'Acquitté Kisembo',
   'location': 'Bunia',
   'status': 'Missing',
   'typeOfDeath': None,
   'charges': None,
   'startDisplay': 'June 26, 2003',
   'mtpage': 'https://cpj.org/data/people/acquitte-kisembo/',
   'country': 'Democratic Republic of the Congo',
   'type': 'Journalist',
   'motiveConfirmed': None},
  {'organizations': 'Al-Sharqiya TV',
   'fullName': 'Ahmed Al-Dulami',
   'location': 'Baiji',
   'status': 'Missing',
   'typeOfDeath': None,
   'charges': None,
   'startDisplay': 'June 1, 2014',
   'mtpage': 'https://cpj.org/data/people/ahmed-al-dulami/',
   'country': 'Iraq',
   'type': 'Journalist',
   'motiveConfirmed': None},
  {'organizations': 'Minivan News',
   'fullName': 'Ahmed Rilwan Abdulla',
   'location': 'Malé',
   'status': 'Missing',
   'typeOfDeath': None,
   'charges': None,
   'startDisplay': 'August 7, 2014',
   'mtpage':

### The steps above are for one seperate page. Below we will do that for all pages

In [6]:
# the first part is the same as above
import requests

headers = {
    'sec-ch-ua': '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Referer': 'https://cpj.org/',
    'sec-ch-ua-mobile': '?0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
    'sec-ch-ua-platform': '"Windows"',
}
# here we set the url as base_url
base_url = 'https://datamanager.cpj.org/api/datamanager/reports/entries'
#and create the params based on what the first url we used has. For example, in the second cell we see that the url is:
#https://datamanager.cpj.org/api/datamanager/reports/entries?distinct(personId)&includes=organizations,fullName,location,status,typeOfDeath,charges,startDisplay,mtpage,country,type,motiveConfirmed&sort=fullName&pageNum=1&pageSize=20&in(status,%27Missing%27)&ge(year,1992)&le(year,2024)
# the first part is the base url (as mentioned)
#then all params are what follows
params = {
    'distinct': 'personId',
    'includes': 'organizations,fullName,location,status,typeOfDeath,charges,startDisplay,mtpage,country,type,motiveConfirmed',
    'sort': '-fullName',
    'pageNum': 1,
    'pageSize': 20,
    'in': 'status,\'Missing\'',
    'ge': 'year,1992',
    'le': 'year,2024'
}

#we create an empty list to store the results
all_results = []
# we need a while loop because we don't know for how many pages is this needed
while True:
    # and we do the same as before, but now for all pages
    response = requests.get(base_url, headers=headers, params=params)
    data = response.json()
    
    # appending the current page of results to the all_results list
    all_results.extend(data.get('data', []))
    
    # checking to see if there are more pages
    current_page = data.get('pageNum', 1)
    total_pages = data.get('pageCount', 1)
    
    if current_page >= total_pages:
        break
    
    # updating the page number to get the next page of results
    params['pageNum'] += 1

# here i print all the results to see if it worked
print(all_results)

[{'organizations': 'Al Barqa TV', 'fullName': 'Younis al-Mabrok al-Moghazy', 'location': 'eastern Libya', 'status': 'Missing', 'typeOfDeath': None, 'charges': None, 'startDisplay': 'August 1, 2014', 'mtpage': 'https://cpj.org/data/people/younis-al-mabrok-al-moghazy/', 'country': 'Libya', 'type': 'Journalist', 'motiveConfirmed': None}, {'organizations': 'Lita-M', 'fullName': 'Yelena Petrova', 'location': 'Chechnya', 'status': 'Missing', 'typeOfDeath': None, 'charges': None, 'startDisplay': 'August 11, 1996', 'mtpage': 'https://cpj.org/data/people/yelena-petrova/', 'country': 'Russia', 'type': 'Journalist', 'motiveConfirmed': None}, {'organizations': 'Aleppo News Network', 'fullName': 'Walid al-Qasim', 'location': 'Aleppo', 'status': 'Missing', 'typeOfDeath': None, 'charges': None, 'startDisplay': 'October 12, 2014', 'mtpage': 'https://cpj.org/data/people/walid-al-qasim/', 'country': 'Syria', 'type': 'Journalist', 'motiveConfirmed': None}, {'organizations': 'al-Arabiya', 'fullName': 'Wah

In [7]:
#I have a list of dictionaries and I store it into a dataframe
df = pd.DataFrame(all_results)
df

,organizations,fullName,location,status,typeOfDeath,charges,startDisplay,mtpage,country,type,motiveConfirmed
0,Al Barqa TV,Younis al-Mabrok al-Moghazy,eastern Libya,Missing,None,None,"August 1, 2014",https://cpj.org/data/people/younis-al-mabrok-a...,Libya,Journalist,None
1,Lita-M,Yelena Petrova,Chechnya,Missing,None,None,"August 11, 1996",https://cpj.org/data/people/yelena-petrova/,Russia,Journalist,None
2,Aleppo News Network,Walid al-Qasim,Aleppo,Missing,None,None,"October 12, 2014",https://cpj.org/data/people/walid-al-qasim/,Syria,Journalist,None
3,al-Arabiya,Waheed Mohammed Naji Haider (Waheed al-Sufi),Sana'a,Missing,None,None,"April 6, 2015",https://cpj.org/data/people/waheed-mohammed-na...,Yemen,Journalist,None
4,Freelance,Vladjimir Legagneur,Port-au-Prince,Missing,None,None,"March 14, 2018",https://cpj.org/data/people/vladjimir-legagneur/,Haiti,Journalist,None
...,...,...,...,...,...,...,...,...,...,...,...
67,El Imparcial,Alfredo Jiménez Mota,Hermosillo,Missing,None,None,"April 2, 2005",https://cpj.org/data/people/alfredo-jimenez-mota/,Mexico,Journalist,None
68,Escenario Calentano,Alan García Aguilar,Arcelia,Missing,None,None,January 2023,https://cpj.org/data/people/alan-garcia-aguilar/,Mexico,Journalist,None
69,Minivan News,Ahmed Rilwan Abdulla,Malé,Missing,None,None,"August 7, 2014",https://cpj.org/data/people/ahmed-rilwan-abdulla/,Maldives,Journalist,None
70,Al-Sharqiya TV,Ahmed Al-Dulami,Baiji,Missing,None,None,"June 1, 2014",https://cpj.org/data/people/ahmed-al-dulami/,Iraq,Journalist,None


In [8]:
#I save my results as a .csv and .xlsx files
df.to_csv("CPJ_missing.csv")
df.to_excel("CPJ_missing.xlsx")